In [7]:
import sys
import valurap
from valurap import rest_client

import imp
imp.reload(rest_client)

c = rest_client.Client()

tgt1 = 230
tgt2 = 230
tgt_hb = 110

c.spp(1, 60000, 0)
c.spp(2, 60000, 0)
c.sfv(ch=1,val=500)
c.sfv(ch=2,val=500)
c.sfv(ch=3,val=1000)
c.spt(ch=1,val=tgt1)
c.spt(ch=2,val=tgt2)
c.spt(ch=3,val=tgt_hb)

print("homing")
c.abort()
c.home()
c.wait_idle()
c.moveto(X1=-190*80, X2=175*80, Y=-16000)
c.wait_idle()

ok1 = 0
ok2 = 0
c.spt(ch=1,val=tgt1)
c.spt(ch=2,val=tgt2)

print("temps approach", tgt1, tgt2)
while True:
    r = c.thermo_state()
    if tgt1 > 0:
        if abs(r["temp1"] - tgt1) < 10 and r["ext1"] > 10 and r["ext1"] < 900:
            ok1 += 1
        else:
            ok1 = 0
    else:
        ok1 = 100
        
    if tgt2 > 0:
        if abs(r["temp2"] - tgt2) < 10 and r["ext2"] > 10 and r["ext2"] < 900:
            ok2 += 1
        else:
            ok2 = 0
    else:
        ok2 = 100
        
    print("{:4d} {:3d} {:2d} | {:4d} {:3d} {:2d} | {:4d} {:3d}".format(
        r["ext1"], r["temp1"], ok1,
        r["ext2"], r["temp2"], ok2,
        r["ext3"], r["temp3"]
    ))
    
    if ok1 > 6 and ok2 > 6:
        break
        
    time.sleep(5)

print("temps tracking", tgt1, tgt2)
c.spp(1, 28000, 400)
c.spp(2, 20000, 350)
ok1 = 0
ok2 = 0
while True:
    r = c.thermo_state()
    if tgt1 > 0:
        if abs(r["temp1"] - tgt1) < 3 and r["ext1"] > 10 and r["ext1"] < 900:
            ok1 += 1
        else:
            ok1 = 0
    else:
        ok1 = 100

    if tgt2 > 0:
        if abs(r["temp2"] - tgt2) < 3 and r["ext2"] > 10 and r["ext2"] < 900:
            ok2 += 1
        else:
            ok2 = 0
    else:
        ok2 = 100
    
    print("{:4d} {:3d} {:2d} | {:4d} {:3d} {:2d} | {:4d} {:3d}".format(
        r["ext1"], r["temp1"], ok1,
        r["ext2"], r["temp2"], ok2,
        r["ext3"], r["temp3"]
    ))

    if ok1 > 23 and ok2 > 23 :
        break
    time.sleep(5)
print("temps ok")
print("prime extruders")

axes = []
if tgt1 > 0:
    axes.append("E1")
if tgt2 > 0:
    axes.append("E2")

c.enable(axes=",".join(axes))

c.move(E1=-50000, E2=-16000)
r = c.wait_idle()
print("all done")


homing
temps approach 230 230
1000 102  0 | 1000 134  0 | 1000  44
1000 109  0 | 1000 142  0 | 1000  45
1000 115  0 | 1000 150  0 | 1000  46
1000 121  0 | 1000 157  0 | 1000  46
1000 127  0 | 1000 165  0 | 1000  47
1000 132  0 | 1000 172  0 | 1000  48
1000 138  0 | 1000 180  0 | 1000  49
1000 144  0 | 1000 187  0 | 1000  49
1000 149  0 | 1000 193  0 | 1000  50
1000 154  0 | 1000 200  0 | 1000  51
1000 160  0 | 1000 206  0 | 1000  51
1000 165  0 | 1000 213  0 | 1000  52
1000 171  0 |  656 219  0 | 1000  52
1000 176  0 |  453 222  1 | 1000  53
1000 181  0 |  389 223  2 | 1000  53
1000 186  0 |  399 223  3 | 1000  54
1000 191  0 |  434 222  4 | 1000  54
1000 196  0 |  450 222  5 | 1000  55
1000 200  0 |  448 222  6 | 1000  56
1000 205  0 |  443 222  7 | 1000  56
1000 209  0 |  438 222  8 | 1000  57
 949 214  0 |  433 222  9 | 1000  57
 692 218  0 |  434 222 10 | 1000  58
 507 221  1 |  434 222 11 | 1000  58
 395 223  2 |  441 222 12 | 1000  59
 360 223  3 |  436 222 13 | 1000  59
 368 223

KeyboardInterrupt: 

In [75]:
axes = []
if tgt1 > 0:
    axes.append("E1")
if tgt2 > 0:
    axes.append("E2")

c.enable(axes=",".join(axes))

c.move(E2=-837/2*100)
r = c.wait_idle()
display(r)

{'idle': True,
 'state': {'apg_states': [{'name': 'E2', 'x': -194852.265625, 'v': 0},
   {'name': 'X2', 'x': 13992.59375, 'v': 0},
   {'name': 'Y', 'x': -16007.4140625, 'v': 0}],
  'pc': 31,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}

In [6]:
c.abort()
c.sfv(ch=1,val=500)
c.sfv(ch=2,val=500)
c.sfv(ch=3,val=1000)
c.spt(ch=1,val=0)
c.spt(ch=2,val=0)
c.spt(ch=3,val=0)


{'ok': 1}

In [ ]:
imp.reload(rest_client)
c = rest_client.Client()

tgts = [100, 150, 200, 230]
r = c.thermo_state()
c.spp(1,60000, 0)
start_tgt = r["target1"]
for tgt in tgts:
    if tgt < start_tgt:
        continue
    print("approach tgt:", tgt)
    c.spt(ch=1,val=tgt)
    ok = 0
    while True:
        r = c.thermo_state()
        if abs(r["temp1"] - tgt) < 10 and r["ext1"] > 10 and r["ext1"]<900:
            ok += 1
        else:
            ok = 0
        print(r["ext1"],r["temp1"], ok)
        if ok > 3:
            break
        time.sleep(10)

print("tracking")
c.spp(1,20000,400)
ok = 0
while True:
    r = c.thermo_state()
    if abs(r["temp1"] - tgt) < 3 and r["ext1"] > 10 and r["ext1"]<900:
        ok += 1
    else:
        ok = 0
    print(r["ext1"],r["temp1"], ok)
    if ok > 5:
        break
    time.sleep(10)


In [ ]:
%matplotlib notebook

import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LinearRegression
from random import randint
from valurap.optozero import calibration


import numpy as np
x1_mtx = np.array([
    [970.87196948,   0.        , 319.39923978],
    [  0.        , 968.54821206, 222.53847797],
    [  0.        ,   0.        ,   1.        ]
])

x1_dist = np.array([[
    1.62236835e-01,
    -6.54970258e-01,
    3.75880870e-03,
    1.38494135e-03,
    -4.52002591e+00
]])


x2_mtx = np.array([
    [966.18776784,   0.        , 310.95960944],
    [  0.        , 965.62677186, 190.37336236],
    [  0.        ,   0.        ,   1.        ]
])

x2_dist = np.array([[
    1.61754264e-01,
    -6.38459089e-01,
    1.63921796e-03,
    -5.25521344e-03,
    -3.23895623e+00
]])

def pattern_coords(X, Y, axe="X1"):
    if axe == "X1":
        xa = -11715
        ya = -5935
        xb = 6374
        yb = 7278
    else:
        xa = -6477
        ya = 1579
        xb = 11610
        yb = 14778

    x0 = xa + (xb - xa) / 7 * X
    y0 = ya + (yb - ya) / 5 * Y
    
    return x0, y0

def move_to_pattern(X, Y, axe="X1", x_offset=0, y_offset=0):
    x0, y0 = pattern_coords(X,Y,axe)
    c.moveto(**{axe: x0 + x_offset, "Y": y0 + y_offset})
    r = wait(axe)
    return r

def get_circles(attempts=4, axe="X1"):
    if axe == "X1":
        cap = cv2.VideoCapture('http://orange:8081/')
    elif axe == "X2":
        cap = cv2.VideoCapture('http://orange:8082/')
    else:
        raise RuntimeError("Unknown axe", axe)
        
    ret, img = cap.read()
    #ret, img = cap.read()
    #ret, img = cap.read()
    cap.release()
    assert img.shape == (480, 640, 3)

    ret, circles = cv2.findCirclesGrid(img, (6,6))
    if ret:
        cv2.drawChessboardCorners(img, (6, 6), circles, ret)
        #display(circles)
        x = circles[:,0][:,0]
        y = circles[:,0][:,1]
        x0 = np.average(x)
        y0 = np.average(y)
        return x0, y0, img
    else:
        if attempts > 0:
            return get_circles(attempts - 1, axe)
    return (None, None, img)

def get_all_circles(attempts=4, axe="X1"):
    if axe == "X1":
        cap = cv2.VideoCapture('http://orange:8081/')
    elif axe == "X2":
        cap = cv2.VideoCapture('http://orange:8082/')
    else:
        raise RuntimeError("Unknown axe", axe)
        
    ret, img = cap.read()
    #ret, img = cap.read()
    #ret, img = cap.read()
    cap.release()
    assert img.shape == (480, 640, 3)

    ret, circles = cv2.findCirclesGrid(img, (6,6))
    if ret:
        return circles
    else:
        if attempts > 0:
            return get_all_circles(attempts - 1, axe)
    return None

def wait(axe="X1"):
    r = c.wait_idle()
    x = None
    y = None
    for s in r["state"]["apg_states"]:
        if s["name"] == axe:
            x = s["x"]
        if s["name"] == "Y":
            y = s["x"]
            
    return x,y

def get_objp():
    param1 = np.zeros((6 * 6, 3), np.float32)
    param2 = (np.mgrid[-3:3,-3:3] + 0.5).T.reshape(-1,2)
    params = [param1, param2]

    objp = params[0]
    objp[:, :2] = params[1]

    return 2.469438 * objp


def optozero(axe="X1", delta=500):
    a = []
    b = []
    cur_x = 0
    cur_y = 0
    dd = 1
    for i in range(45):
        if i == 20:
            print("optozero: set to 2.5x step")
            dd = 2.5
        elif i == 30:
            print("optozero: set to 5x step")
            dd = 5

        tx = randint(-delta * dd, delta * dd)
        ty = randint(-delta * dd, delta * dd)
        dx = tx - cur_x
        dy = ty - cur_y
        cur_x = tx
        cur_y = ty
        
        c.move(**{axe: dx, "Y": dy})
        r = wait(axe)

        cx, cy, img = get_circles(axe=axe)
        if cx is not None:
            a.append([cx, cy])
            b.append(r)
            if len(a) >= 8:
                break

    if len(a) < 4:
        print("Failed to find target pattern", len(a))
        return None
    
    a = np.array(a)
    b = np.array(b)

    x = b[:,0]
    y = b[:,1]

    reg_x = LinearRegression().fit(a, x)
    reg_y = LinearRegression().fit(a, y)
    tx = reg_x.predict([[320, 240]])[0]
    ty = reg_y.predict([[320, 240]])[0]

    c.moveto(**{axe: int(round(tx)), "Y": int(round(ty))})
    rf = wait(axe)
    return rf


In [ ]:
#c.moveto(X1=1000)
#c.moveto(X2=1000)
base1_x = -4020
base1_y = -9578
base2_x = 1205
base2_y = -2069

c.home()
c.wait_idle()
c.moveto(X1=0)
c.moveto(Y=0)
c.wait_idle()
c.moveto(Z=100000)
c.wait_idle()

c.moveto(X1=base1_x, X2=base2_x, Y=base1_y)
c.wait_idle()
first1_x, first1_y = optozero("X1", delta=50)
display((first1_x, first1_y))

c.moveto(X1=base1_x, X2=base2_x, Y=base2_y)
c.wait_idle()
first2_x, first2_y = optozero("X2", delta=50)
display((first2_x, first2_y))

c.moveto(Z=50000)

for i in range(5):
    c.moveto(X1=first1_x, X2=first2_x, Y=first1_y)
    c.wait_idle()
    r = optozero("X1", delta=50)
    display(r)

    c.moveto(X1=first1_x, X2=first2_x, Y=first2_y)
    c.wait_idle()
    r = optozero("X2", delta=50)
    display(r)


In [ ]:
(-4028.01171875, -9764.01171875)
(1247.9921875, -2236.0078125)
(-4027.01171875, -9765.0078125)
(1242.9921875, -2226.0078125)
(-4026.01171875, -9770.0078125)
(1243.9921875, -2231.0078125)
(-4023.01171875, -9763.01171875)
(1243.98828125, -2227.0078125)
(-4027.01171875, -9764.0078125)
(1247.99609375, -2230.00390625)

(-4000.01171875, -9459.01171875)
(1209.9921875, -1959.01171875)
(-3996.01171875, -9462.0078125)
(1215.9921875, -1934.01171875)
(-3997.01171875, -9466.01171875)
(1213.9921875, -1956.0078125)
(-4000.01171875, -9468.01171875)
(1218.9921875, -1939.0078125)
(-4001.015625, -9474.015625)
(1216.98828125, -1959.0078125)

(-4013.01171875, -9746.01171875)
(1220.9921875, -2207.0078125)
(-4016.0078125, -9748.0078125)
(1219.98828125, -2207.0078125)
(-4016.0078125, -9747.0078125)
(1219.9921875, -2207.0078125)
(-4015.0078125, -9746.0078125)
(1220.9921875, -2208.0078125)
(-4017.0078125, -9748.0078125)
(1220.9921875, -2208.00390625)

(-4029.01171875, -9765.015625)
(1232.9921875, -2225.01171875)
(-4030.015625, -9764.015625)
(1233.9921875, -2226.0078125)
(-4032.01171875, -9765.01171875)
(1232.98828125, -2225.01171875)
(-4032.01171875, -9766.01171875)
(1232.98828125, -2225.01171875)
(-4030.01171875, -9766.0078125)
(1231.98828125, -2225.0078125)

In [ ]:
%matplotlib notebook

import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LinearRegression
from random import randint




fig=plt.figure(figsize=(8, 8))
rows = 6
columns = 6

axe = "X2"

c.moveto(Z=100000)
c.moveto(X1=0, X2=0, Y=0)
wait(axe)

if axe == "X1":
    c.moveto(X1=0, X2=170*80, Y=0)
else:
    c.moveto(X1=-190*80, X2=0, Y=0)
wait(axe)

x0, y0 = pattern_coords(3, 3, axe)

    
a = []
b = []
for i in range(6):
    x1 = x0 + randint(-500, 500)
    y1 = y0 + randint(-500, 500)

    c.moveto(**{axe: x1, "Y": y1})
    r = wait(axe)

    cx, cy, img = get_circles(axe=axe)
    if cx is not None:
        a.append([cx, cy])
        b.append(r)

    fig.add_subplot(rows, columns, i+1)    
    plt.imshow(img)

a = np.array(a)
b = np.array(b)
display(a)
display(b)

x = b[:,0]
y = b[:,1]

reg_x = LinearRegression().fit(a, x)
reg_y = LinearRegression().fit(a, y)
display(reg_x.coef_)
display(reg_x.intercept_)
display(reg_y.coef_)
display(reg_y.intercept_)
tx = reg_x.predict([[320, 240]])[0]
ty = reg_y.predict([[320, 240]])[0]
display(tx, ty)

c.moveto(**{axe: int(round(tx)), "Y": int(round(ty))})
rf = wait(axe)
xf, yf, img = get_circles(axe=axe)
display(xf, yf)
fig.add_subplot(rows, columns, i+2)    
plt.imshow(img)


In [ ]:
import cv2
import numpy as np


objp = get_objp()
print(objp)

objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
real_state = []  # motion state

axe = "X2"
c.moveto(Z=100000)
c.moveto(X1=0, X2=0, Y=0)
wait(axe)

if axe == "X1":
    xa = -11715
    ya = -5935
    xb = 6374
    yb = 7278
    c.moveto(X1=0, X2=170*80, Y=0)
    wait(axe)

else:
    xa = -6477
    ya = 1579
    xb = 11610
    yb = 14778
    c.moveto(X1=-190*80, X2=0, Y=0)
    wait(axe)

T = (3, 3)
xT0 = xa + (xb - xa) / 7 * T[0]
yT0 = ya + (yb - ya) / 5 * T[1]

T = (1, 1)
xT1 = xa + (xb - xa) / 7 * T[0]
yT1 = ya + (yb - ya) / 5 * T[1]

T = (3, 1)
xT2 = xa + (xb - xa) / 7 * T[0]
yT2 = ya + (yb - ya) / 5 * T[1]

T = (5, 1)
xT3 = xa + (xb - xa) / 7 * T[0]
yT3 = ya + (yb - ya) / 5 * T[1]

for z in range(100000, 19000, -7000):
    c.moveto(Z=z)
    for i in range(10):
        for x0, y0, k in [
            [xT0, yT0, 3],
            [xT1, yT1, 1],
            [xT2, yT2, 1],
            [xT3, yT3, 1],
        ]:
            x1 = x0 + randint(-500*k, 500*k)
            y1 = y0 + randint(-500*k, 500*k)

            c.moveto(**{axe: x1, "Y": y1})
            r = wait(axe)

            circles = get_all_circles(axe=axe)
            if circles is not None:
                objpoints.append(objp)
                imgpoints.append(circles)
                real_state.append(r)



In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1][1:3], None, None)


In [ ]:
mtx

In [ ]:
dist

In [ ]:
x1_calibration= {
    "objpoints": objpoints,
    "imgpoints": imgpoints,
    "shape": img.shape[::-1][1:3],
    "mtx": mtx,
    "dist": dist,
    "rvecs": rvecs,
    "tvecs": tvecs,
    "real_state": real_state
}


In [ ]:
import pickle
#pickle.dump(x1_calibration, open("x2_calib.pickle", "wb"))

In [ ]:
results = []

for axe in ["X1", "X2"]:
    c.moveto(Z=50000)
    c.moveto(X1=-190*80, X2=170*80, Y=0)

    move_to_pattern(4,3,axe=axe)
    r = optozero(axe, delta=100)
    if r is None:
        print("optozero failed")
        continue

    c.moveto(Z=30000)
    r = optozero(axe, delta=50)
    if r is None:
        print("optozero failed")
        continue

    objp = get_objp()

    a = 150
    for dx, dy in [
        [None, 0],
        [a, a],
        [-2 * a, 0],
        [0, -2 * a],
        [2 * a, 0]
    ]:
        if dx is not None:
            c.move(**{axe: dx, "Y": dy})
            r = wait(axe)

        for z in range(15000, 100000, 1600*5):
            c.moveto(Z=z)
            r = wait(axe)

            for i in range(3):
                circles = get_all_circles(axe=axe)
                if circles is None:
                    print(z, "no circles")
                    continue

                retval, rvec, tvec = cv2.solvePnP(
                    objp, circles,
                    calibration[axe]["mtx"], calibration[axe]["dist"])
                if not retval:
                    print(z, "solvePnP failed")
                    continue

                print(z, retval, list(tvec.T[0]))
                results.append((axe, z, r, rvec, tvec, circles))


In [ ]:
pickle.dump(results, open('bed_leveling_final.pickle', "wb"))

In [ ]:
import pickle 
c1 = pickle.load(open('height_calibration_T4_3.pickle', "rb"))
c2 = pickle.load(open('height_calibration_T4_4.pickle', "rb"))
c3 = pickle.load(open('height_calibration_T5_3.pickle', "rb"))
c4 = pickle.load(open('height_calibration_T5_4.pickle', "rb"))

c_all = c1 + c2 + c3 + c4

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np

axe = []
x = []
y = []
z = []
rvec_x = []
rvec_y = []
rvec_z = []
tvec_x = []
tvec_y = []
tvec_z = []
for c in c_all:
    axe_, z_, (x_, y_), rvec, tvec, circles = c
    x_ = int(round(x_))
    y_ = int(round(y_))
    axe.append(axe_)
    x.append(x_)
    y.append(y_)
    z.append(z_)
    rvec_x.append(rvec[0][0])
    rvec_y.append(rvec[1][0])
    rvec_z.append(rvec[2][0])
    tvec_x.append(tvec[0][0])
    tvec_y.append(tvec[1][0])
    tvec_z.append(tvec[2][0])
    

df = pd.DataFrame({"axe": axe, "x": x, "y": y, "z":z, 
                   "rvec_x": rvec_x, "rvec_y": rvec_y, "rvec_z": rvec_z,
                   "tvec_x": tvec_x, "tvec_y": tvec_y, "tvec_z": tvec_z,
                  })

is_x1 = df["x"] * 0
is_x1[df["axe"] == "X1"] = 1.0
df["is_x1"] = is_x1
df["x_c"] = df["x"] / 80.0 + df["tvec_x"] + df["is_x1"] * 70
df["y_c"] = df["y"] / 80.0 + df["tvec_y"] + df["is_x1"] * 95

df["z_c"] = df["z"] / 1600.0 * (1.107 - 0.022 * df["is_x1"]) - df["tvec_z"]

df.plot(x="x", y="z_c", kind='scatter', style="*")

In [ ]:
c.moveto(Z=50000)
c.moveto(X1=-190*80, X2=170*80, Y=0)

objp = get_objp()
results = []
for axe in ["X1", "X2"]:
    for tx in range(8):
        for ty in range(6):
            c.moveto(Z=50000)
            move_to_pattern(tx, ty, axe=axe)
            r = optozero(axe, delta=100)
            if r is None:
                print("optozero failed")
                continue

            c.moveto(Z=30000)
            r = optozero(axe, delta=50)
            if r is None:
                print("optozero failed")
                continue

            a = 100
            cdx = 0
            cdy = 0
            cdz = 0
            for dx, dy, dz in [
                [0, 0, 0],
                [a, a, 0],
                #[-2 * a, 0, 0],
                #[0, -2 * a, 0],
                #[2 * a, 0, 0],
                [0, 0, 1600],
                #[a, a, 0],
                [-2 * a, -2 * a, 0],
                #[0, -2 * a, 0],
                #[2 * a, 0, 0],
            ]:
                cdx += dx
                cdy += dy
                cdz += dz
                
                c.move(**{axe: dx, "Y": dy, "Z": dz})
                r = wait(axe)

                circles = get_all_circles(axe=axe)
                if circles is None:
                    print(tx, ty, cdx, cdy, cdz, "no circles")
                    continue

                retval, rvec, tvec = cv2.solvePnP(
                    objp, circles,
                    calibration[axe]["mtx"], calibration[axe]["dist"])
                if not retval:
                    print(tx, ty, cdx, cdy, cdz, "solvePnP failed")
                    continue

                print(tx, ty, cdx, cdy, cdz, tvec.T[0][2])
                results.append((axe, r, tx, ty, cdx, cdy, cdz, rvec, tvec, circles))


In [ ]:
import pandas as pd
import numpy as np

res = pickle.load(open('bed_leveling_final.pickle', "rb"))

axe = []
x = []
y = []
tx = []
ty = []
cdx = []
cdy = []
cdz = []
rvec_x = []
rvec_y = []
rvec_z = []
tvec_x = []
tvec_y = []
tvec_z = []
for r in res:
    axe_, (x_, y_), tx_, ty_, cdx_, cdy_, cdz_, rvec, tvec, circles = r
    x_ = int(round(x_))
    y_ = int(round(y_))
    axe.append(axe_)
    x.append(x_)
    y.append(y_)
    tx.append(tx_)
    ty.append(ty_)
    cdx.append(cdx_)
    cdy.append(cdy_)
    cdz.append(cdz_)
    rvec_x.append(rvec[0][0])
    rvec_y.append(rvec[1][0])
    rvec_z.append(rvec[2][0])
    tvec_x.append(tvec[0][0])
    tvec_y.append(tvec[1][0])
    tvec_z.append(tvec[2][0])
    

df = pd.DataFrame({"axe": axe,
                   "x": x, "y": y,
                   "tx": tx, "ty": ty,
                   "cdx": cdx, "cdy": cdy, "cdz": cdz,
                   "rvec_x": rvec_x, "rvec_y": rvec_y, "rvec_z": rvec_z,
                   "tvec_x": tvec_x, "tvec_y": tvec_y, "tvec_z": tvec_z,
                  })

df["x_c"] = df["x"] + df["cdx"] + df["tvec_z"]
df["y_c"] = df["y"] + df["cdy"] + df["tvec_z"]
df_x1 = df[
    np.logical_and(
        np.logical_and(df.axe == "X1", df.cdx==0), 
        np.logical_and(df.cdy == 0, df.cdz == 0)
    )
][["tx", "ty", "tvec_z"]].copy()
avg_x1 = df_x1["tvec_z"].mean()
df_x1["dz"] = df_x1.tvec_z - avg_x1

df_x2 = df[
    np.logical_and(
        np.logical_and(df.axe == "X2", df.cdx==0), 
        np.logical_and(df.cdy == 0, df.cdz == 0)
    )
][["tx", "ty", "tvec_z"]].copy()
avg_x2 = df_x2["tvec_z"].mean()
df_x2["dz"] = df_x2.tvec_z - avg_x2

#display(df_x1)
#display(df_x2)
df_m = df_x1.merge(df_x2, on=["tx", "ty"], how="outer")
df_m[["tx", "ty", "dz_x", "dz_y"]]

In [ ]:
import pickle

c.home()
r = c.wait_idle()
print(r)

c.moveto(X1=-190*80, X2=170*80)
r1 = c.wait_idle()
print(r1)

c.moveto(Y=0, Z=10000)
r2 = c.wait_idle()
print(r2)



In [13]:
from valurap import path_planning2 as pp2
import valurap
from valurap import rest_client
import time
import pickle
import os.path

import imp
imp.reload(rest_client)
imp.reload(pp2)

c = rest_client.Client(emu=False)

class Prn():
    offsets = {
        "E1": {
            "X": 0,
            "Y": 0,
            "Z": 5040 - 1600*0.8 #+ 50000
        },
        "E2": {
            "X": 7.0 * 80,
            "Y": (120 + 2)*80,
            "Z": 9110 - 1600*0 #+ 50000
        }
    }
    def __init__(self, c):
        self.c = c
        self.current_X1 = None
        self.current_X2 = None
        self.current_Y = None
        self.current_Z = None
        self.current_extruder = None
        self.planner = pp2.PathPlanner()
        self.accumulated_layers = []
        
    def process_job_end(self):
        print("[PRN] DONE")

    def process_home(self):
        assert not self.accumulated_layers
        print("[PRN] HOME")
        c = self.c
        c.abort()
        time.sleep(2)
        c.home()
        r = c.wait_idle()
        print(r)

        c.moveto(X1=-190*self.planner.spm, X2=170*self.planner.spm)
        rr1 = self.c.wait_idle()
        print(rr1)

        c.moveto(Y=0, Z=20000)
        rr2 = c.wait_idle()
        print(rr2)
        
        prn.set_state(rr1)
        prn.set_state(rr2)
        c.enable(axes="E1,E2")
        c.move(E1=-50000, E2=-16000)
        r = c.wait_idle()


    def process_move(self, x=None, y=None, z=None, e=None):   
        c_e = self.current_extruder
        mapping = {"Y": "Y"}
        segs = []
        
        axe_x = "X1"
        if x is not None:
            real_x = x*self.planner.spm + self.offsets[c_e]["X"]
            if self.current_extruder == "E1":
                dx = real_x - self.current_X1
                self.current_X1 = real_x
            else:
                axe_x = "X2"
                dx = real_x - self.current_X2
                self.current_X2 = real_x

            print("[PRN] MOVE {} TO {} ({})".format(axe_x, real_x, dx))

            segs += self.planner.ext_to_code(dx / self.planner.spm, 100, axe=axe_x)
            
        mapping[axe_x] = "X"

        if y is not None:
            real_y = y*self.planner.spm + self.offsets[c_e]["Y"]
            dy = real_y - self.current_Y
            segs += self.planner.ext_to_code(dy/self.planner.spm, 100, axe="Y")
            print("[PRN] MOVE Y TO {} ({})".format(real_y, dy))
            self.current_Y = real_y

        axe_z = "Z"
        if z is not None:
            real_z = z*self.planner.spmz + self.offsets[c_e]["Z"]
            dz = real_z - self.current_Z
            print("[PRN] MOVE Z TO {} ({}) ()mm".format(real_z, dz, z))
        
            segs += self.planner.ext_to_code(dz/self.planner.spmz, 3, axe="Z")
            self.current_Z = real_z
            assert e is None
        elif e is not None:
            axe_z = c_e
            segs += self.planner.ext_to_code(e, 10, axe=c_e)
            print("[PRN] MOVE {} for {}mm".format(c_e, e))
            
        mapping[axe_z] = "Z"

        tupled_segment = []
        for dt, segs in segs:
            tupled_segment.append((dt, tuple([s.to_tuple() for s in segs])))

        self.accumulated_layers.append(("segment", {
            "map": mapping,
            "acc_step": self.planner.acc_step,
            "extruder": self.current_extruder
        }, tupled_segment))
        
    def process_segment(self, segment, start, end, e):
        assert e == self.current_extruder
        if self.current_extruder == "E1":
            current_x = self.current_X1
        else:
            current_x = self.current_X2
            
        if start:
            print("Expected start", start)
            real_x = start[0]*self.planner.spm + self.offsets[e]["X"]
            real_y = start[1]*self.planner.spm + self.offsets[e]["Y"]
            assert abs(current_x - real_x) < 0.001
            assert abs(self.current_Y - real_y) < 0.001
        print("[PRN] DO SEGMENT", segment[1], len(segment[2]), len(layer_data))
        real_x = end[1][0]*self.planner.spm + self.offsets[e]["X"]
        real_y = end[1][1]*self.planner.spm + self.offsets[e]["Y"]
        if self.current_extruder == "E1":
            self.current_X1 = real_x
        else:
            self.current_X2 = real_x
        self.current_Y = real_y

        self.accumulated_layers.append(segment)
        while True:
            if self.c.emu:
                break
            r = self.c.state()
            #print(r)
            if r["state"]["code_len"] < 1000000:
                print("submit code")
                break
            #print("code_len is too big: {}, sleeping".format(r["state"]["code_len"]))
            time.sleep(1)
        self.c.exec_code(self.accumulated_layers)
        self.accumulated_layers = []

    def set_state(self, r):
        if self.c.emu:
            self.current_X1, self.current_X2, self.current_Y, self.current_Z = (
                -15200.4140625, 13598.58984375, -0.41015625, 19951.015625
            )
            return
            
        for st in r["state"]["apg_states"]:
            if st["name"] == "X1":
                self.current_X1 = st["x"]
            if st["name"] == "X2":
                self.current_X2 = st["x"]
            if st["name"] == "Y":
                self.current_Y = st["x"]
            if st["name"] == "Z":
                self.current_Z = st["x"]

        #print(self.current_X1, self.current_X2, self.current_Y, self.current_Z)
        
prn = Prn(c)

fn_tpl = 'box20_void_dual_{:05d}.layer'
state = "init"
i = -1
while True:
    i += 1
    fn = fn_tpl.format(i)
    
    while True:
        if os.path.exists(fn):
            break
        print("layer {} is not yet ready".format(fn))
        time.sleep(2)
        
    layer_data = open(fn, 'rb').read()
    layer = pickle.loads(layer_data)
    print("layer:", fn, len(layer))
    start = None
    end = None
    do_home = False
    switch_extruder = None
    segment = None
    
    expected_x = None
    expected_y = None
    expected_z = None
    expected_extruder = None

    for l in layer:
        cmd = l[0]
        print(" cmd:", cmd, state)
        if cmd == "do_home":
            do_home = True
            print("  p:", l)
        elif cmd == "start":
            start = l
            print("  p:", l[3], l[4], l[1])
        elif cmd == "end_state":
            end = l
            print("  p:", l[1])
        elif cmd == "segment":
            assert segment is None
            segment = l
            print("  len: ", len(l[2]))
        elif cmd == "extruder_switch":
            assert segment is None
            switch_extruder = l[1]
            print("  ext: ", l[1])
            
    if state == "init":
        if switch_extruder is not None:
            assert do_home == False and segment is None
            prn.current_extruder = "E{}".format(switch_extruder+1)
        elif do_home:
            assert len(segment[2]) == 8
            prn.process_home()
            state = "homed"
        else:
            assert False
        continue
    elif state == "homed":
        assert start[3] == (None, None, 0)
        assert segment
        expected_x = start[1]["X"]
        expected_y = start[1]["Y"]
        expected_z = start[1]["Z"]
        expected_extruder = start[4]
        state = "working"
        prn.process_move(x=expected_x, y=expected_y, z=expected_z)
        prn.process_segment(segment, None, end, expected_extruder)
    elif state == "working":
        if do_home:
            prn.process_job_end()
            break
        elif switch_extruder is not None:
            print("[PRN] SWITCH TO", switch_extruder)
            if prn.current_extruder == "E1":
                target_x = -180
            else:
                target_x = 165
            prn.process_move(x=target_x)
            prn.current_extruder = "E{}".format(switch_extruder+1)
            state = "switched"
        else:
            assert start[3]
            expected_z = start[1]["Z"]
            print("Z:", expected_z)
            expected_extruder = start[4]
            exp_start = (start[3][0], start[3][1], start[3][2])
            prn.process_move(z=expected_z)
            prn.process_segment(segment, exp_start, end, expected_extruder)
    elif state == "switched":
        print(start[3])
        assert start[3][:2] == (None, None)
        assert segment
        expected_x = start[1]["X"]
        expected_y = start[1]["Y"]
        expected_z = start[1]["Z"]
        expected_extruder = start[4]
        prn.process_move(z=expected_z)
        prn.process_move(y=expected_y)
        prn.process_move(x=expected_x)
        prn.process_segment(segment, None, end, expected_extruder)
        state = "working"
    else:
        assert False
        
while True:
    r = c.state()
    if r["idle"]:
        break
    print(r["state"]["code_len"])
    time.sleep(10)
    
c.abort()
c.home()
c.wait_idle()
c.abort()
c.spt(ch=1, val=0)
c.spt(ch=2, val=0)
c.spt(ch=3, val=0)


layer: box20_void_dual_00000.layer 2
 cmd: extruder_switch init
  ext:  0
 cmd: end_state init
  p: (None, None, 0)
layer: box20_void_dual_00001.layer 3
 cmd: do_home init
  p: ('do_home', {'X': 158.75, 'Y': -250.0, 'Z': 12.5, 'E': 0.0})
 cmd: segment init
  len:  8
 cmd: end_state init
  p: (None, None, 0)
[PRN] HOME
{'idle': True, 'state': {'apg_states': [{'name': 'X1', 'x': 12698.84765625, 'v': 0}, {'name': 'X2', 'x': -13501.1484375, 'v': 0}, {'name': 'Y', 'x': -20001.1484375, 'v': 0}], 'pc': 51, 'free_buf': None, 'code_len': 0, 'busy': 0}, 'abs_safe': True}
{'idle': True, 'state': {'apg_states': [{'name': 'X1', 'x': -15200.4140625, 'v': 0}, {'name': 'X2', 'x': 13599.58984375, 'v': 0}, {'name': 'Y', 'x': -20001.1484375, 'v': 0}], 'pc': 41, 'free_buf': None, 'code_len': 0, 'busy': 0}, 'abs_safe': True}
{'idle': True, 'state': {'apg_states': [{'name': 'X2', 'x': 13598.58984375, 'v': 0}, {'name': 'Y', 'x': -0.41015625, 'v': 0}, {'name': 'Z', 'x': 19951.015625, 'v': 0}], 'pc': 41, 'free

submit code
layer: box20_void_dual_00018.layer 3
 cmd: extruder_switch working
  ext:  0
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 1356405)
[PRN] SWITCH TO 0
[PRN] MOVE X2 TO 13760.0 (12791.675728390459)
layer: box20_void_dual_00019.layer 3
 cmd: start switched
  p: (None, None, 1356405) E1 {'X': 6.9590000000000245, 'Y': -6.958999999999991, 'Z': 1.8000000000000007}
 cmd: segment switched
  len:  339
 cmd: end_state switched
  p: (-7.003061362690642, -5.99965314331057, 1446644)
(None, None, 1356405)
[PRN] MOVE Z TO 6640.000000000001 (-5349.999999999999) ()mm
[PRN] MOVE Y TO -556.7199999999992 (-9909.214357323273)
[PRN] MOVE X1 TO 556.720000000002 (14956.720000000001)
[PRN] DO SEGMENT {'map': {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}, 'acc_step': 10000, 'extruder': 'E1'} 339 24533
submit code
layer: box20_void_dual_00020.layer 3
 cmd: start working
  p: (-7.003061362690642, -5.99965314331057, 1446644) E1 {'X': -6.999999999999977, 'Y': -5.99999999999999, 'Z': 2.1000

layer: box20_void_dual_00038.layer 3
 cmd: start working
  p: (-7.003061362690642, -5.99965314331057, 2424800) E1 {'X': -6.999999999999977, 'Y': -5.99999999999999, 'Z': 3.9000000000000004}
 cmd: segment working
  len:  633
 cmd: end_state working
  p: (8.99472829195729, 9.115077138910419, 2551349)
Z: 3.9000000000000004
[PRN] MOVE Z TO 10000.0 (480.0) ()mm
Expected start (-7.003061362690642, -5.99965314331057, 2424800)
[PRN] DO SEGMENT {'map': {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}, 'acc_step': 10000, 'extruder': 'E1'} 633 45958
submit code
layer: box20_void_dual_00039.layer 3
 cmd: extruder_switch working
  ext:  1
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 2551349)
[PRN] SWITCH TO 1
[PRN] MOVE X1 TO -14400 (-15119.578263356583)
layer: box20_void_dual_00040.layer 3
 cmd: start switched
  p: (None, None, 2551349) E2 {'X': -2.666999999999975, 'Y': 5.1000000000000085, 'Z': 3.9000000000000004}
 cmd: segment switched
  len:  84
 cmd: end_state switched
  p: (-2.6662

layer: box20_void_dual_00057.layer 3
 cmd: extruder_switch working
  ext:  1
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 3529505)
[PRN] SWITCH TO 1
[PRN] MOVE X1 TO -14400 (-15119.578263356583)
layer: box20_void_dual_00058.layer 3
 cmd: start switched
  p: (None, None, 3529505) E2 {'X': -2.666999999999975, 'Y': 5.1000000000000085, 'Z': 5.700000000000001}
 cmd: segment switched
  len:  84
 cmd: end_state switched
  p: (-2.6662751836527603, 5.099958294210955, 3580903)
(None, None, 3529505)
[PRN] MOVE Z TO 18230.0 (5349.999999999998) ()mm
[PRN] MOVE Y TO 10168.0 (9438.793828887166)
[PRN] MOVE X2 TO 346.64000000000203 (-13413.359999999999)
[PRN] DO SEGMENT {'map': {'X2': 'X', 'Y': 'Y', 'E2': 'Z'}, 'acc_step': 10000, 'extruder': 'E2'} 84 4573
submit code
layer: box20_void_dual_00059.layer 3
 cmd: start working
  p: (-2.6662751836527603, 5.099958294210955, 3580903) E2 {'X': -2.666999999999975, 'Y': 5.1000000000000085, 'Z': 6.000000000000001}
 cmd: segment workin

layer: box20_void_dual_00078.layer 3
 cmd: extruder_switch working
  ext:  0
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 4616925)
[PRN] SWITCH TO 0
[PRN] MOVE X2 TO 13760.0 (12791.675728390459)
layer: box20_void_dual_00079.layer 3
 cmd: start switched
  p: (None, None, 4616925) E1 {'X': 6.9590000000000245, 'Y': -6.958999999999991, 'Z': 7.800000000000001}
 cmd: segment switched
  len:  339
 cmd: end_state switched
  p: (-7.003061362690642, -5.99965314331057, 4707164)
(None, None, 4616925)
[PRN] MOVE Z TO 16240.000000000002 (-5349.999999999998) ()mm
[PRN] MOVE Y TO -556.7199999999992 (-9909.214357323273)
[PRN] MOVE X1 TO 556.720000000002 (14956.720000000001)
[PRN] DO SEGMENT {'map': {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}, 'acc_step': 10000, 'extruder': 'E1'} 339 24533
submit code
layer: box20_void_dual_00080.layer 3
 cmd: start working
  p: (-7.003061362690642, -5.99965314331057, 4707164) E1 {'X': -6.999999999999977, 'Y': -5.99999999999999, 'Z': 8.100000000000001}

...

layer: box20_void_dual_00117.layer 3
 cmd: extruder_switch working
  ext:  1
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 6790025)
[PRN] SWITCH TO 1
[PRN] MOVE X1 TO -14400 (-15119.578263356583)
layer: box20_void_dual_00118.layer 3
 cmd: start switched
  p: (None, None, 6790025) E2 {'X': -2.666999999999975, 'Y': 5.1000000000000085, 'Z': 11.700000000000001}
 cmd: segment switched
  len:  84
 cmd: end_state switched
  p: (-2.6662751836527603, 5.099958294210955, 6841423)
(None, None, 6790025)
[PRN] MOVE Z TO 27830.0 (5350.0) ()mm
[PRN] MOVE Y TO 10168.0 (9438.793828887166)
[PRN] MOVE X2 TO 346.64000000000203 (-13413.359999999999)
[PRN] DO SEGMENT {'map': {'X2': 'X', 'Y': 'Y', 'E2': 'Z'}, 'acc_step': 10000, 'extruder': 'E2'} 84 4573
submit code
layer: box20_void_dual_00119.layer 3
 cmd: start working
  p: (-2.6662751836527603, 5.099958294210955, 6841423) E2 {'X': -2.666999999999975, 'Y': 5.1000000000000085, 'Z': 12.000000000000002}
 cmd: segment working
  len: 

layer: box20_void_dual_00138.layer 3
 cmd: extruder_switch working
  ext:  0
 cmd: segment working
  len:  4
 cmd: end_state working
  p: (None, None, 7877445)
[PRN] SWITCH TO 0
[PRN] MOVE X2 TO 13760.0 (12791.675728390459)
layer: box20_void_dual_00139.layer 3
 cmd: start switched
  p: (None, None, 7877445) E1 {'X': 6.9590000000000245, 'Y': -6.958999999999991, 'Z': 13.800000000000002}
 cmd: segment switched
  len:  339
 cmd: end_state switched
  p: (-7.003061362690642, -5.99965314331057, 7967684)
(None, None, 7877445)
[PRN] MOVE Z TO 25840.000000000004 (-5350.0) ()mm
[PRN] MOVE Y TO -556.7199999999992 (-9909.214357323273)
[PRN] MOVE X1 TO 556.720000000002 (14956.720000000001)
[PRN] DO SEGMENT {'map': {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}, 'acc_step': 10000, 'extruder': 'E1'} 339 24533
submit code
layer: box20_void_dual_00140.layer 3
 cmd: start working
  p: (-7.003061362690642, -5.99965314331057, 7967684) E1 {'X': -6.999999999999977, 'Y': -5.99999999999999, 'Z': 14.100000000000001}
 cmd: se

layer: box20_void_dual_00158.layer 3
 cmd: start working
  p: (-7.003061362690642, -5.99965314331057, 8945840) E1 {'X': -6.999999999999977, 'Y': -5.99999999999999, 'Z': 15.900000000000002}
 cmd: segment working
  len:  633
 cmd: end_state working
  p: (8.99472829195729, 9.115077138910419, 9072389)
Z: 15.900000000000002
[PRN] MOVE Z TO 29200.000000000004 (480.0) ()mm
Expected start (-7.003061362690642, -5.99965314331057, 8945840)
[PRN] DO SEGMENT {'map': {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}, 'acc_step': 10000, 'extruder': 'E1'} 633 45958
submit code
layer: box20_void_dual_00159.layer 3
 cmd: start working
  p: (8.99472829195729, 9.115077138910419, 9072389) E1 {'X': 9.000000000000025, 'Y': 9.11800000000001, 'Z': 16.200000000000003}
 cmd: segment working
  len:  942
 cmd: end_state working
  p: (-6.021152008822537, -6.439612837941968, 9290208)
Z: 16.200000000000003
[PRN] MOVE Z TO 29680.000000000004 (480.0) ()mm
Expected start (8.99472829195729, 9.115077138910419, 9072389)
[PRN] DO SEGMENT {'

334554
332804
329360
324765
323937
320280
317763
314290
310686
309867
306286
303635
299050
297419
296600
292928
290472
285180
284364
280636
278882
275545
271106
270292
266666
264866
260415
257765
256947
253512
250837
246217
244466
242651
239847
237316
231670
230860
227370
225556
221579
218083
217269
213406
210740
207231
204630
203815
200495
197890
192427
191608
187872
187027
183623
178366
177551
173901
172164
167629
164978
164163
160627
157985
153455
151666
149705
147219
143822
138790
137908
135197
133539
129067
125572
124762
120457
118692
114675
111336
111336
107048
104514
99860
98237
95681
93916
90506
85865
85050
81509
79827
75661
72205
66905
64358
59071
56508
52137
48524
45123
40496
36252
32012
27722
24090
20425
15389
12627
10061
6487
4006
2327
0
0
0


{'ok': 1}

In [ ]:
while True:
    r = c.state()
    if r["idle"]:
        break
    print(r["state"]["code_len"])
    time.sleep(10)
    
c.home()
c.wait_idle()
c.abort()
c.spt(ch=1, val=0)
c.spt(ch=2, val=0)
c.spt(ch=3, val=0)

In [31]:
c.abort()
c.home()
c.wait_idle()
c.moveto(Y=0 + 120*80)
c.wait_idle()
c.moveto(X2=0)
c.wait_idle()
#c.move(Z=-3*1600)
#c.wait_idle()

{'idle': True, 'state': {'apg_states': [{'name': 'X1', 'x': 12698.84765625, 'v': 0}, {'name': 'X2', 'x': -13501.1484375, 'v': 0}, {'name': 'Y', 'x': -20001.1484375, 'v': 0}], 'pc': 51, 'free_buf': None, 'code_len': 0, 'busy': 0}, 'abs_safe': True}


{'idle': True,
 'state': {'apg_states': [{'name': 'X1', 'x': 12698.84765625, 'v': 0},
   {'name': 'X2', 'x': -0.41015625, 'v': 0},
   {'name': 'Y', 'x': 9599.5859375, 'v': 0}],
  'pc': 31,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}

In [43]:
c.moveto(Z=20000)
r = c.wait_idle()

c.moveto(Z=9110 + 0.0*1600)
r = c.wait_idle()

display(r)

{'idle': True,
 'state': {'apg_states': [{'name': 'X2', 'x': -1.41015625, 'v': 0},
   {'name': 'Y', 'x': 9598.5859375, 'v': 0},
   {'name': 'Z', 'x': 9108.68359375, 'v': 0}],
  'pc': 31,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}

In [46]:
c.moveto(Z=20000)
r = c.wait_idle()

c.moveto(Z=9110 + 0.3*1600)
r = c.wait_idle()

c.move(X2=-50*80, E2=-837)
r = c.wait_idle()

display(r)

{'idle': True,
 'state': {'apg_states': [{'name': 'E2', 'x': -838.00390625, 'v': 0},
   {'name': 'X2', 'x': 3740.35546875, 'v': 0},
   {'name': 'Z', 'x': 9588.6953125, 'v': 0}],
  'pc': 41,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}

In [60]:
if 0:
    c.enable(axes="E2")
    c.move(E2=-10000)
    r = c.wait_idle()

#c.move(X2=20*80)
#r = c.wait_idle()
k=-5
k1 = abs(k)
c.move(X2=50*80*k, E2=-837*k1)
r = c.wait_idle()

display(r)

{'idle': True,
 'state': {'apg_states': [{'name': 'E2', 'x': -63482.33203125, 'v': 0},
   {'name': 'X2', 'x': -6688.9921875, 'v': 0},
   {'name': 'Z', 'x': 9587.6953125, 'v': 0}],
  'pc': 41,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}